# Convert SQLite Database for Carrot2 
 * Settings

In [ ]:
import json
import os
%run assets/func_inputoutput.py
# Setup paths
cwd = os.getcwd()
print("Current working directory:", cwd)
# Define pdf file directory
pdf_collection_path = 'eGRASP'
pdf_path = os.path.join(cwd, "docs", pdf_collection_path)
# Define database directory
base_name = 'eGRASP.db'#'LigaseE3_7.db'
database_name = os.path.join(cwd, "docs", pdf_collection_path, base_name)
# Create settings dictionary
settings = {
    'working_directory': cwd,
    'pdf_collection_path': pdf_collection_path,
    'pdf_path': pdf_path,
    'base_name': base_name,
    'database_name': database_name,
    'additional': 'value1'  # 
}
# Save settings to a file
save_settings(settings)
# Later on, load settings and unpack directly into variables
working_directory, pdf_collection_path, pdf_path, base_name, database_name, remaining_settings = load_settings()
# Print the loaded settings
print(f'\nworking_directory {working_directory}')
print(f"\nPDF Collection Path: {pdf_collection_path}")
print(f"\npdf_path: {pdf_path}")  
print(f"\nbase_name: {base_name}")  
print(f"\ndatabase_name: {database_name}")  
print(f"\nRemaining Settings: {remaining_settings}")

# Chose Sqlite Database discoverd in subdirectories in the root 

In [2]:
import sqlite3
import json
import os
import sys
import glob

# --- ipywidgets Imports ---
import ipywidgets as widgets
from IPython.display import display, clear_output
# ------------------------

# --- Configuration for Record Counting ---
# Option 1: Set this if you know the exact table name in all DBs
# TARGET_TABLE_NAME = 'YourSpecificTableName'
# Option 2: Set to None to dynamically find the *first* table in each DB
TARGET_TABLE_NAME = None
# ------------------------------------------

# Assume %run assets/func_inputoutput.py defines load_settings
try:
    %run assets/func_inputoutput.py
except Exception as e:
    print(f"Warning: Could not run func_inputoutput.py: {e}")
    # def load_settings(): return (os.getcwd(), '', '', '', '', {}) # Dummy if needed

# --- Helper function to get record count ---
def get_record_count(db_path, table_name=None):
    """Safely connects to an SQLite DB and counts records in a table."""
    count = None
    conn = None
    cursor = None
    resolved_table_name = table_name
    target_table_for_msg = "first available" # Default message part

    if not os.path.isfile(db_path):
        return None # File doesn't exist

    try:
        # Use file URI and read-only mode for safety
        conn = sqlite3.connect(f'file:{db_path}?mode=ro', uri=True, timeout=5) # Added timeout
        cursor = conn.cursor()

        # If table name not provided, find the first one
        if not resolved_table_name:
            # Query common metadata tables first, then fallback
            # Sometimes user tables might be listed after system tables
            cursor.execute("""
                SELECT name FROM sqlite_master
                WHERE type='table' AND name NOT LIKE 'sqlite_%'
                ORDER BY name
                LIMIT 1;
            """)
            table_result = cursor.fetchone()
            if table_result:
                resolved_table_name = table_result[0]
            else:
                # Fallback if no non-sqlite tables found
                cursor.execute("SELECT name FROM sqlite_master WHERE type='table' LIMIT 1;")
                table_result = cursor.fetchone()
                if table_result:
                    resolved_table_name = table_result[0]
                    print(f"Info: Found system table '{resolved_table_name}' in {os.path.basename(db_path)} as first table.")
                else:
                     print(f"Warning: No tables found in {os.path.basename(db_path)}.")
                     return 0 # No tables found

        # Update message part if table name was resolved dynamically
        if not table_name and resolved_table_name:
             target_table_for_msg = f"'{resolved_table_name}' (first)"

        if resolved_table_name:
            # *** FIX: Build SQL string without f-string for quoting ***
            # Escape double quotes within the table name itself
            escaped_table_name = resolved_table_name.replace("\"", "\"\"")
            # Construct the final SQL query string
            sql = f'SELECT COUNT(*) FROM "{escaped_table_name}"'
            # *** End FIX ***

            cursor.execute(sql)
            count_result = cursor.fetchone()
            if count_result:
                count = count_result[0]
            else:
                 count = 0 # Should not happen with COUNT(*) unless error

    except sqlite3.OperationalError as e:
        # Catch specific errors like "database is locked" or "no such table"
        print(f"Warning: Operational error querying {os.path.basename(db_path)} ({target_table_for_msg}): {e}")
        count = None
    except sqlite3.DatabaseError as e:
         # Catch other potential DB errors (e.g., corrupted file)
         print(f"Warning: Database error accessing {os.path.basename(db_path)}: {e}")
         count = None
    except Exception as e:
         # Catch any other unexpected errors during DB access
         print(f"Warning: Unexpected error processing {os.path.basename(db_path)}: {e}")
         count = None
    finally:
        # Ensure resources are closed even if errors occurred
        if cursor:
            try: cursor.close()
            except Exception: pass # Ignore errors during close
        if conn:
            try: conn.close()
            except Exception: pass # Ignore errors during close

    return count

# --- Initial Setup ---
cwd = os.getcwd()
print("Current working directory:", cwd)

default_pdf_collection_path = 'eGRASP'
default_base_name = 'eGRASP.db'
database_name_default = os.path.join(cwd, "docs", default_pdf_collection_path, default_base_name)

# --- Load Settings ---
database_name_default_effective = database_name_default
working_directory = cwd
try:
    settings_data = load_settings()
    working_directory = settings_data[0]
    database_name_from_file = settings_data[4]
    print("\nLoaded settings from file.")
    if not database_name_from_file or not os.path.isfile(database_name_from_file):
        print(f"Warning: DB from settings ('{database_name_from_file}') invalid/missing.")
        print(f"Falling back to initial default: {database_name_default_effective}")
    else:
        database_name_default_effective = database_name_from_file
        print(f"Using default database from settings file: {database_name_default_effective}")
except Exception as e:
     print(f"\nCould not load settings: {e}. Using initial defaults.")

# --- Find Database Files Recursively ---
db_search_dir = os.path.join(working_directory, "docs")
print(f"\nRecursively searching for databases in and under: {db_search_dir}")

all_potential_db_paths = []
if os.path.isdir(db_search_dir):
    patterns = ["*.db", "*.sqlite", "*.sqlite3"]
    db_files = []
    for pattern in patterns:
        recursive_pattern = os.path.join(db_search_dir, '**', pattern)
        found = glob.glob(recursive_pattern, recursive=True)
        db_files.extend(found)
    all_potential_db_paths = sorted(list(set(db_files)))
    print(f"Total unique potential DB files found by glob: {len(all_potential_db_paths)}")
else:
    print(f"Error: Search directory does not exist: {db_search_dir}")

# Ensure default is included in the list to process, even if outside search dir
if os.path.isfile(database_name_default_effective) and database_name_default_effective not in all_potential_db_paths:
    all_potential_db_paths.append(database_name_default_effective)
    print(f"Added default DB ({os.path.basename(database_name_default_effective)}) to list as it was outside search path.")


# --- Create Dropdown Options WITH Counts ---
print("Getting record counts for found databases...")
db_options_final = {} # Dictionary: Display Name [Count] -> Full Path
default_value = None  # Full path of the actual default to pre-select

# Process all potential paths to build the options dictionary
for db_path in all_potential_db_paths:
    if not os.path.isfile(db_path): # Skip if somehow a non-file got listed
         continue

    # Get record count
    record_count = get_record_count(db_path, TARGET_TABLE_NAME)
    count_str = f"[{record_count} records]" if record_count is not None else "[Count Error]"

    # Determine Display Name (relative path or basename)
    display_base = ""
    is_default = (db_path == database_name_default_effective)
    try:
        # Prefer relative path from search directory
        relative_path = os.path.relpath(db_path, db_search_dir)
        display_base = relative_path
    except ValueError:
        # Fallback (e.g., different drive or outside search dir)
        display_base = os.path.basename(db_path)

    # Construct final display name
    if is_default:
        display_name = f"(Default) {display_base} {count_str}"
        default_value = db_path # Set the default value for the dropdown
    else:
        display_name = f"{display_base} {count_str}"

    # Add to final options map
    db_options_final[display_name] = db_path

# Ensure default_value is set if the default DB was missing but found later
if default_value is None and database_name_default_effective in all_potential_db_paths:
     default_value = database_name_default_effective

# Fallback default value if the effective default doesn't exist *at all*
if default_value is None and db_options_final:
    default_value = list(db_options_final.values())[0]


# --- Define Global State Variable ---
final_selected_database_name = [default_value]

# --- Widgets ---
if not db_options_final:
     print("ERROR: No valid database files found or specified.")
else:
    # Sort options alphabetically for display (optional)
    # Note: This sorts by the display string including count and (Default)
    sorted_options = dict(sorted(db_options_final.items()))

    db_dropdown = widgets.Dropdown(
        options=sorted_options, # Use sorted options
        value=default_value,  # Pre-select the correct default full path
        description='Select DB:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='auto', min_width='400px') # Increased min_width
    )
    confirm_button = widgets.Button(
        description="Use Selected Database", button_style='info', icon='check'
    )
    output_widget = widgets.Output()

    # --- Button Click Action ---
    def on_confirm_button_clicked(b):
        with output_widget:
            clear_output(wait=True)
            selected_path = db_dropdown.value # This is the full path
            if selected_path and os.path.isfile(selected_path):
                final_selected_database_name[0] = selected_path # Update shared state
                selected_display = [k for k, v in db_options_final.items() if v == selected_path]
                print(f"Confirmed selection: {selected_display[0] if selected_display else 'Unknown'}")
                print(f"--> Ready to run next cell using this database.")
            else:
                print(f"Error: Selected path '{selected_path}' is not valid.")
                print(f"Keeping previous selection: {final_selected_database_name[0]}")

    confirm_button.on_click(on_confirm_button_clicked)

    # --- Display Widgets ---
    ui = widgets.VBox([db_dropdown, confirm_button, output_widget])
    print("--- Please select the database below and click 'Use Selected Database' ---")
    display(ui)

Current working directory: e:\users\behnisch\python\envs\GPT\openai-quickstart-node-master\PDF_screen\Langchain\LangChain-Chat-with-Your-Data-main
Settings loaded from settings.json.

path and name settings from above were loaded

  There are additional settings that could be added:
  additional: value1

Loaded settings from file.
Using default database from settings file: e:\users\behnisch\python\envs\GPT\openai-quickstart-node-master\PDF_screen\Langchain\LangChain-Chat-with-Your-Data-main\docs\eGRASP\eGRASP.db

Recursively searching for databases in and under: e:\users\behnisch\python\envs\GPT\openai-quickstart-node-master\PDF_screen\Langchain\LangChain-Chat-with-Your-Data-main\docs
Total unique potential DB files found by glob: 21
Getting record counts for found databases...
--- Please select the database below and click 'Use Selected Database' ---


# Convert the Sqlite database into a json file for import and text clustering in Carrot2
* https://search.carrot2.org/#/workbench

In [3]:
import sqlite3
import json
import os
import sys

# --- CRITICAL: Get the selected database path from the previous cell ---
# Check if the state variable exists from the previous cell execution
if 'final_selected_database_name' not in locals() or not final_selected_database_name:
    print("ERROR: Database selection variable not found. Please run the previous cell and select a database.")
    # Optionally raise an error: raise RuntimeError("Database not selected")
    database_name = None # Indicate failure
else:
    database_name = final_selected_database_name[0] # Get the path stored in the list

# --- Proceed only if a valid database was selected ---
if database_name and os.path.isfile(database_name):
    print(f"\n--- Starting Export Process ---")
    print(f"Using selected database: {database_name}")

    # Define Output JSON Path (can be defined here or loaded if needed)
    OUTPUT_JSON_FILENAME = 'carrot2_input.json'
    # Use current working directory or define explicitly
    output_json_path = os.path.join(os.getcwd(), OUTPUT_JSON_FILENAME)
    print(f"Output JSON will be saved to: {output_json_path}")


    # --- Database Connection and Data Fetching ---
    conn = None
    cursor = None
    results = []
    try:
        print(f"Connecting to selected database: {database_name}")
        conn = sqlite3.connect(database_name)
        cursor = conn.cursor()
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
        tables = cursor.fetchall()
        if not tables:
            raise ValueError("No tables found in the database.")
        table_name = tables[0][0]
        print(f"Using table: {table_name}")

        # Define columns to fetch
        db_columns_to_fetch = ['ID', 'Title', 'Abstract', 'Body'] # Add 'DOI' if needed

        # Verify columns exist
        cursor.execute(f"PRAGMA table_info({table_name});")
        available_columns = [info[1] for info in cursor.fetchall()]
        columns_to_select = []
        for col in db_columns_to_fetch:
            if col in available_columns:
                columns_to_select.append(col)
            else:
                 print(f"Warning: Column '{col}' not found in table '{table_name}'. Skipping.")

        if not columns_to_select:
             raise ValueError(f"None of the required columns found in table '{table_name}'.")

        column_string = ", ".join([f'"{col}"' for col in columns_to_select])
        query = f"SELECT {column_string} FROM {table_name}"

        print(f"Executing query: {query}")
        cursor.execute(query)
        results = cursor.fetchall()
        print(f"Fetched {len(results)} records.")

    except Exception as e:
        print(f"\nDatabase error: {e}")
        # Handle error appropriately, maybe skip JSON export
        results = [] # Ensure results is empty on error
    finally:
        if cursor: cursor.close()
        if conn:
             conn.close()
             print("Database connection closed.")

    # --- JSON Export Section ---
    if not results:
        print("No records fetched or error occurred. JSON file will not be created.")
    else:
        print(f"\nStarting JSON export to: {output_json_path}")
        documents_for_json = []
        col_map = {name: index for index, name in enumerate(columns_to_select)}

        for result_row in results:
            # Extract data using map
            record_id = result_row[col_map.get('ID')] if 'ID' in col_map else None
            title = result_row[col_map.get('Title')] if 'Title' in col_map else None
            abstract_db = result_row[col_map.get('Abstract')] if 'Abstract' in col_map else None
            body_db = result_row[col_map.get('Body')] if 'Body' in col_map else None
            # doi_db = result_row[col_map.get('DOI')] if 'DOI' in col_map else None

            # Convert None to Empty String / ID to String
            id_str = str(record_id) if record_id is not None else ""
            title_str = title if title is not None else ""
            abstract_str = abstract_db if abstract_db is not None else ""
            body_str = body_db if body_db is not None else ""
            doi_str = "" # doi_db if doi_db is not None else ""

            # Create dictionary with separate 'abstract' and 'body'
            doc_dict = {
                'file_id': id_str,
                'title': title_str,
                'doi': doi_str,
                'abstract': abstract_str,
                'body': body_str
            }
            documents_for_json.append(doc_dict)

        # Write the list of dictionaries to a JSON file
        try:
            with open(output_json_path, 'w', encoding='utf-8') as jsonfile:
                json.dump(documents_for_json, jsonfile, indent=2, ensure_ascii=False)
            print(f"Successfully written {len(documents_for_json)} documents to {output_json_path}")
        except Exception as e:
            print(f"\nError writing JSON file: {e}")

    print("\nScript finished.")

else:
    print("ERROR: No valid database selected in the previous step. Export aborted.")


--- Starting Export Process ---
Using selected database: e:\users\behnisch\python\envs\GPT\openai-quickstart-node-master\PDF_screen\Langchain\LangChain-Chat-with-Your-Data-main\docs\LigaseE3 - Copy\LigaseE3_7.db
Output JSON will be saved to: e:\users\behnisch\python\envs\GPT\openai-quickstart-node-master\PDF_screen\Langchain\LangChain-Chat-with-Your-Data-main\carrot2_input.json
Connecting to selected database: e:\users\behnisch\python\envs\GPT\openai-quickstart-node-master\PDF_screen\Langchain\LangChain-Chat-with-Your-Data-main\docs\LigaseE3 - Copy\LigaseE3_7.db
Using table: document_table
Executing query: SELECT "ID", "Title", "Abstract", "Body" FROM document_table
Fetched 94 records.
Database connection closed.

Starting JSON export to: e:\users\behnisch\python\envs\GPT\openai-quickstart-node-master\PDF_screen\Langchain\LangChain-Chat-with-Your-Data-main\carrot2_input.json
Successfully written 94 documents to e:\users\behnisch\python\envs\GPT\openai-quickstart-node-master\PDF_screen

# take the carrot2_input.json and import into Carrot2 for clustering and tree or pi-chart